# Preparation

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import cv2 as cv
import tensorflowjs as tfjs

In [16]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [17]:
NUM_CLASSES = 10
BATCH_SIZE = 32
IMG_SIZE = (128, 128)

In [18]:
data = tf.keras.utils.image_dataset_from_directory(
    'data',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

Found 36006 files belonging to 10 classes.


In [19]:
# Data preprocessing

# Inject variation into the data
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
])

def preprocess(images, labels):
    images = tf.cast(images, tf.float32) / 255.0 # Normalize pixel values to be between 0 and 1 
    images = data_augmentation(images) # Inject variation into the data
    labels = tf.one_hot(labels, NUM_CLASSES) # Convert to one-hot encoding
    return images, labels

data = data.map(preprocess)

In [20]:
# Data splitting
train_size = int(0.8 * len(data))
test_size = int(0.1 * len(data))
val_size = int(0.1 * len(data))

train_data = data.take(train_size)
test_data = data.skip(train_size).take(test_size)
val_data = data.skip(train_size + test_size).take(val_size)

# Models

## MobileNetV2

In [21]:
# Create model using transfer learning
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)

# Unfreeze the top layers for fine-tuning
for layer in base_model.layers[-30:]:
    layer.trainable = True

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

# Compile model with a lower learning rate for fine-tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        filepath='models/MobileNetV2.keras',
        monitor='val_accuracy',
        save_best_only=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=2
    ),
    tf.keras.callbacks.TensorBoard(log_dir="logs")
]

# Training

In [22]:
# Train the model
history = model.fit(
    train_data,
    epochs=10,
    validation_data=val_data,
    callbacks=callbacks
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_data)
print(f"\nTest accuracy: {test_accuracy:.4f}")

Epoch 1/10
900/900 [==============================] - 99s 105ms/step - loss: 0.8835 - accuracy: 0.7361 - val_loss: 0.6055 - val_accuracy: 0.8200 - lr: 1.0000e-04
Epoch 2/10
900/900 [==============================] - 93s 104ms/step - loss: 0.4844 - accuracy: 0.8417 - val_loss: 0.4043 - val_accuracy: 0.8619 - lr: 1.0000e-04
Epoch 3/10
900/900 [==============================] - 93s 104ms/step - loss: 0.3651 - accuracy: 0.8814 - val_loss: 0.4232 - val_accuracy: 0.8661 - lr: 1.0000e-04
Epoch 4/10
900/900 [==============================] - 94s 105ms/step - loss: 0.2891 - accuracy: 0.9030 - val_loss: 0.4185 - val_accuracy: 0.8608 - lr: 1.0000e-04
Epoch 5/10
900/900 [==============================] - 93s 103ms/step - loss: 0.1906 - accuracy: 0.9348 - val_loss: 0.3225 - val_accuracy: 0.8895 - lr: 2.0000e-05
Epoch 6/10
900/900 [==============================] - 94s 105ms/step - loss: 0.1504 - accuracy: 0.9498 - val_loss: 0.3297 - val_accuracy: 0.8945 - lr: 2.0000e-05
Epoch 7/10
900/900 [========

In [5]:
# Save model in TensorFlow.js format
model = tf.keras.models.load_model('models/MobileNetV2.keras')
tfjs.converters.save_keras_model(model, 'frontend/public/models/MobileNetV2_tfjs')